In [1]:
import numpy as np
import matplotlib.pyplot as plt

from synthlearners import Synth
from synthlearners.simulator import (
    SimulationConfig,
    PanelSimulator,
    FactorDGP,
)
from synthlearners.plotting import SynthPlotter

## Basic use with Panel Matrices

In [8]:
config = SimulationConfig(
    N=100,
    T=50,
    T_pre=40,
    n_treated=10,
    selection_mean=1.0,
    treatment_effect=0.5,
    dgp=FactorDGP(K=2, sigma=1, time_fac_lb=-0.1, time_fac_ub=0.1, trend_sigma=0.0001),
)
simulator = PanelSimulator(config)
Y, Y_0, L, treated_units = simulator.simulate()

In [9]:
treated_units

array([89, 32, 13, 80, 85, 28, 82, 45,  9, 64])

In [10]:
from synthlearners.mcnnm import MatrixCompletionEstimator

In [11]:
# Create a mask indicating which entries are observed.
mask = np.ones_like(Y)
missing_periods = config.T-config.T_pre

# For treated units, suppose the outcomes in the last N periods are missing.
mask[treated_units, -missing_periods:] = 0  

In [13]:
mask[treated_units[0]]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [14]:
estimator = MatrixCompletionEstimator(
    lambda_param=0.5,
    tau=1.0,
    max_iter=500,
    tol=1e-4,
    verbose=True
)
estimator.fit(Y, mask)
Y_completed = estimator.completed_matrix_

In [19]:
# Counterfactual Values for imputed periods for single treated example
Y_0[treated_units[0], -missing_periods:]

array([-0.76488391, -0.15174888, -0.13119737,  0.30966483,  0.12721515,
        0.81209996,  0.97510635, -1.01615275,  0.85462799, -1.3490142 ])

In [29]:
# Observed Value for imputed periods from single treated example
Y[treated_units[0], -missing_periods:]

array([-0.26488391,  0.34825112,  0.36880263,  0.80966483,  0.62721515,
        1.31209996,  1.47510635, -0.51615275,  1.35462799, -0.8490142 ])

In [30]:
# MC Values for imputed periods for single treated example
Y_completed[treated_units[0], -missing_periods:]

array([ 0.10312538, -0.28523968, -0.33110528,  0.34821213,  0.27433178,
        0.23849457,  0.051028  , -0.34374433,  0.30512483, -0.75447681])

In [35]:
baseline_bias = Y_completed[treated_units[0], -missing_periods:]-Y_0[treated_units[0], -missing_periods:]
print('Baseline Bias of MC value relative to actual coutnerfactual',np.mean(bias))

Baseline Bias of MC value relative to actual coutnerfactual -0.005996657049492582


In [36]:
te = Y[treated_units[0], -missing_periods:]-Y_completed[treated_units[0], -missing_periods:]
print('Estimated Treatment Effect of Observed relative to MC Value',np.mean(te))

Estimated Treatment Effect of Observed relative to MC Value 0.5059966570494925


In [37]:
print('Treatment Effect Bias',np.abs(np.mean(te)-config.treatment_effect))

Treatment Effect Bias 0.005996657049492482
